In [2]:
import snowflake.connector
import requests
import base64


In [14]:
# Snowflake credentials
SNOWFLAKE_ACCOUNT=''
SNOWFLAKE_USER=''
SNOWFLAKE_PASSWORD=''
SNOWFLAKE_DATABASE=''
SNOWFLAKE_SCHEMA=''
SNOWFLAKE_WAREHOUSE=''
SNOWFLAKE_ROLE=''
SNOWFLAKE_TABLE=''

In [24]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

# Fetch records from the table
cursor = conn.cursor()
cursor.execute(f"SELECT IMAGE_URL FROM {SNOWFLAKE_TABLE}")
records = cursor.fetchall()

In [45]:
url = records[0][0]

In [33]:
response = requests.get(records[0][0])
if response.status_code == 200:
    base64_image = base64.b64encode(response.content).decode('utf-8')

In [47]:
import requests
import base64
from PIL import Image
from io import BytesIO

def download_compress_encode(url, quality=50):
    """Download an image, compress it, and convert to Base64."""
    response = requests.get(url)
    
    if response.status_code == 200:
        # Open image from response content
        img = Image.open(BytesIO(response.content))
        
        # Convert to RGB to avoid issues with PNG mode
        img = img.convert("RGB")

        # Compress the image
        buffer = BytesIO()
        img.save(buffer, format="JPEG", quality=quality) 
        
        # Encode to Base64
        base64_encoded = base64.b64encode(buffer.getvalue()).decode("utf-8")
        return base64_encoded
    else:
        raise Exception(f"Failed to download image. Status Code: {response.status_code}")


image_url = records[0][0] 
base64_image = download_compress_encode(image_url, quality=50)

print("Compressed Base64 Image:", base64_image[:100] + "...") 


Compressed Base64 Image: /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABALDA4MChAODQ4SERATGCgaGBYWGDEjJR0oOjM9PDkzODdASFxOQERXRTc4UG1RV19i...


In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('mistral-large', 'You will be provided with an image in base64 format which is related to an event. Describe the image concisely, focusing only on event-specific details. If the image includes information such as the event name, timing, date, or price, extract and provide those details as well. Image URL : ');


In [49]:
import json

# Construct the prompt
prompt = (
    "You will be provided with an image in base64 format which is related to an event. "
    "Describe the image concisely, focusing only on event-specific details. "
    "If the image includes information such as the event name, timing, date, or price, "
    "extract and provide those details as well. "
    f"Base64 Image: {base64_image}"
)

# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('jamba-1.5-large', PARSE_JSON(%s))
"""

cursor.execute(query, (json.dumps({"prompt": prompt}),))


ProgrammingError: 100351 (P0000): Request failed for external function COMPLETE$V2 with remote service error: 400 '"max tokens of 220000 exceeded"
'; requests batch-id: 01bab2de-0004-ab89-0003-d563001aa98e:1:1:0:0; request batch size: 1 rows; request retries: 0; response time (last retry): 2m40.11921s

In [89]:
prompt1 = (
    "You will be provided with an image in base64 format which is related to an event. "
    "Describe the image concisely, focusing only on event-specific details. "
    "If the image includes information such as the event name, timing, date, or price, "
    "extract and provide those details as well. "
    f"Base64 Image: {base64_image}"
)

In [85]:
prompt

'You will be provided with an image url which is related to an event. Describe the image concisely, focusing only on event-specific details. If the image includes information such as the event name, timing, date, or price, extract and provide those details as well. Image URL: https://media.thebostoncalendar.com/images/q_auto,fl_lossy/v1738974124/duz2jd9y4kr285bdamux/v-for-vault-an-electronic-dance-music-valentines.png'

In [93]:
import tiktoken

def count_tokens(text, model="gpt-4o-mini"):
    """Counts the number of tokens in the text using OpenAI-compatible tokenizer."""
    enc = tiktoken.encoding_for_model(model) 
    return len(enc.encode(text))


In [95]:
prompt_text = (
    "You will be provided with an image in base64 format which is related to an event. "
    "Describe the image concisely, focusing only on event-specific details. "
    "If the image includes information such as the event name, timing, date, or price, "
    "extract and provide those details as well. "
    f"Base64 Image: {base64_image}"
)

num_tokens = count_tokens(prompt_text)
print(f"Number of tokens in the prompt: {num_tokens}")

Number of tokens in the prompt: 1312280


In [91]:
def estimate_tokens(text):
    """Roughly estimate tokens based on word count (1.3 words ≈ 1 token)."""
    words = text.split()
    return int(len(words) / 1.3)

num_tokens = estimate_tokens(prompt1)
print(f"Estimated number of tokens: {num_tokens}")


Estimated number of tokens: 37


In [ ]:
from PIL import Image
from io import BytesIO
import base64

def compress_and_encode_image(image_path, quality=50):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        buffer = BytesIO()
        img.save(buffer, format="JPEG", quality=quality) 
        return base64.b64encode(buffer.getvalue()).decode("utf-8")

base64_image = compress_and_encode_image("your_image.jpg")


In [51]:
image_url

'https://media.thebostoncalendar.com/images/q_auto,fl_lossy/v1738974124/duz2jd9y4kr285bdamux/v-for-vault-an-electronic-dance-music-valentines.png'

In [ ]:
##JAMBA

In [ ]:
import json

# Construct the prompt
prompt = (
    "You will be provided with an image url which is related to an event. "
    "Describe the image concisely, focusing only on event-specific details. "
    "If the image includes information such as the event name, timing, date, or price, "
    "extract and provide those details as well. "
    f"Image URL: {image_url}"
)

# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('jamba-1.5-large', PARSE_JSON(%s))
"""

# Execute query safely with parameters
cursor.execute(query, (json.dumps({"prompt": prompt}),))


In [58]:
cursor.fetchone()

(" V-for-Vault: An Electronic Dance Music Valentine's event on February 10, 2024, from 10:00 PM to 2:00 AM at The Grand, Boston. Tickets priced at $15.",)

In [ ]:
##REKA-FLASH

In [60]:
# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('reka-flash', PARSE_JSON(%s))
"""

# Execute query safely with parameters
cursor.execute(query, (json.dumps({"prompt": prompt}),))
cursor.fetchone()


(" **Event: V for Vault - an Electronic Dance Music Valentine's party**\n**Visual Elements:**\n- The image features a dark, edgy aesthetic with neon accents.\n- Central imagery includes a stylized 'V' and heart symbols, suggestive of Valentine's Day.\n- Text overlays indicate the event type ('Electronic Dance Music') and the thematic focus ('Valentines').\n- No specific date, time, or price is visible in the image.\n\n ",)

In [ ]:
##MISTRAL

In [66]:
# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('mistral-large2', PARSE_JSON(%s))
"""

# Execute query safely with parameters
cursor.execute(query, (json.dumps({"prompt": prompt}),))
cursor.fetchone()

(' The image is a promotional poster for an event titled "V for Vault: An Electronic Dance Music Valentine\'s." The event features a DJ lineup including DJs like DJ Pup Dawg, DJ B3t$y, and DJ D-Lux. The event is scheduled for February 10, 2024, from 9 PM to 2 AM. The location is not specified in the image.',)

In [ ]:
##LLAMA

In [68]:
# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('llama3-8b', PARSE_JSON(%s))
"""

# Execute query safely with parameters
cursor.execute(query, (json.dumps({"prompt": prompt}),))
cursor.fetchone()

('The image is a promotional poster for an event called "V for Vault: An Electronic Dance Music Valentine\'s". The event appears to be a music festival or party, likely taking place on Valentine\'s Day. The background of the image is a dark purple color with a subtle pattern of hearts and circuit boards. The main attraction, "V for Vault", is prominently displayed in bold, neon-lit letters. The image also includes a tagline "Get ready to fall in love with the music" and a date "February 14th".',)

In [ ]:
##MIXTRAL

In [70]:
# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('mixtral-8x7b', PARSE_JSON(%s))
"""

# Execute query safely with parameters
cursor.execute(query, (json.dumps({"prompt": prompt}),))
cursor.fetchone()

(' The image is an advertisement for an electronic dance music (EDM) event called "V for Vault: An Electronic Dance Music Valentine\'s." The event is taking place on Valentine\'s Day, which is on a Tuesday in February. The exact date is not provided, but based on the name of the event, it is likely to be on February 14th. The image features a stylized heart with a "V" in the center, and the text "V for Vault" is written in a bold, futuristic font. The background of the image is a dark, metallic color, giving it a modern and high-tech feel. There is no information about the price or location of the event in the image.',)

In [ ]:
##llama3.3-70b

In [72]:
# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('llama3.3-70b', PARSE_JSON(%s))
"""
# Execute query safely with parameters
cursor.execute(query, (json.dumps({"prompt": prompt}),))
cursor.fetchone()

('The image appears to be a promotional poster for an electronic dance music (EDM) event called "V for Vault" on Valentine\'s Day. The event details extracted from the image are:\n\n* Event name: V for Vault\n* Theme: Electronic Dance Music (EDM) on Valentine\'s Day\n* Date: Valentine\'s Day (no specific year mentioned)\n\nNo timing or price information is visible in the image.',)

In [74]:
# Convert input to JSON format
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE('gemma-7b', PARSE_JSON(%s))
"""
# Execute query safely with parameters
cursor.execute(query, (json.dumps({"prompt": prompt}),))
cursor.fetchone()

("## Image Description\n\nThe image is for an electronic dance music (EDM) Valentine's event at a club in Boston. The main focus of the image is the energetic crowd dancing on the floor. The atmosphere is vibrant and festive, with glow sticks and colorful lights.\n\n**Event-specific details:**\n\n* **Event name:** V for Vault - An Electronic Dance Music Valentine's\n* **Timing:** Unknown (not included in the text)\n* **Date:** Unknown (not included in the text)\n* **Price:** Unknown (not included in the text)",)

In [ ]:
##OpenAI

In [79]:
from openai import OpenAI

In [81]:
client = OpenAI(api_key='')
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "You will be provided with an image related to an event. Describe the image concisely, focusing only on event-specific details. If the image includes information such as the event name, timing, date, or price, extract and provide those details. Exclude any unrelated observations or interpretations.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)

In [83]:
response.choices[0].message.content

'**Event Details:**\n\n- **Event Name:** V FOR VAULT\n- **Date:** February 14, 2025\n- **Time:** 7 PM - 1 AM\n- **Location:** The Neal Rantoul Vault Theater\n- **Music By:** Mary AK, D Torrance, Mr. Eyeslee, Dinoblunt 3D Mung Bandit\n- **Age Restriction:** 21+\n- **Ticket Prices:** \n  - $10 Early Bird\n  - $12 General\n  - $15 Door'

In [101]:
client = OpenAI(api_key='')

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "You will be provided with an image related to an event. Describe the image concisely, focusing only on event-specific details. If the image includes information such as the event name, timing, date, or price, extract and provide those details. Exclude any unrelated observations or interpretations.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": "https://media.thebostoncalendar.com/images/q_auto,fl_lossy/v1738974028/qoh1j0ettai2gexekxyx/artz-underground-presents-for-the-love-of-rnb.jpg"},
                },
            ],
        }
    ],
)


In [103]:
response.choices[0].message.content

'**Event Details:**\n\n- **Event Name:** For the Love of RNB\n- **Performers:** Dreion, Ashanti Eva Davenport, Bia Javier, and more\n- **Date:** February 15th\n- **Time:** 7 PM\n- **Price:** $20 online / $25 at the door\n- **Location:** Neal Rantoul Vault Theatre (Lynn Arts), 25 Exchange St., Lynn, MA 01901'